In [ ]:
import pandas as pd
import numpy as np
import json
import os
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelBinarizer
import re
from matplotlib import pyplot as plt

In [ ]:
base_path='/content/gdrive/MyDrive/AmazonReviewsClassifier'

data_list=[]
f1 =open(os.path.join(base_path,'1reviews.json'))
data_list.append(json.load(f1))

f2= open(os.path.join(base_path,'2reviews.json'))
# data_list.append(json.load(f2))

f3=open(os.path.join(base_path,'3reviews.json'))
# data_list.append(json.load(f3))

f4=open(os.path.join(base_path,'4reviews.json'))
data_list.append(json.load(f4))

f5=open(os.path.join(base_path,'5reviews.json'))
data_list.append(json.load(f5))




In [ ]:
##loading the json scraped from website into csv
data = data_list[-1]
data=np.array(data)
b=data.ravel()
df_base=pd.DataFrame.from_dict(list(b))
df_base['rating']=[5.0 for x in range(df_base.shape[0])]

for index in range(len(data_list)-1):
        data=data_list[index]
        data=np.array(data)
        b=data.ravel()
        df=pd.DataFrame.from_dict(list(b))
        df_base=pd.concat([df_base,df])
        

print(df_base.shape)
print(df_base.info())

(3000, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rating  3000 non-null   float64
 1   title   3000 non-null   object 
 2   body    3000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 93.8+ KB
None


In [ ]:
## adding 2 and 3 reviews file also
## due to different size than others, we converted these two ratings .json file using online converter
df_2=pd.read_csv(os.path.join(base_path,'2reviews.csv'))
df_2=df_2[['title','body']]
df_3=pd.read_csv(os.path.join(base_path,'3reviews.csv'))
df_3=df_2[['title','body']]

df_2['rating']=[2 for x in range(df_2.shape[0])]
df_3['rating']=[3 for x in range(df_3.shape[0])]

print(df_2.shape)
print(df_3.shape)
df=pd.concat([df_2,df_3])
df_base=pd.concat([df_base,df],axis=0)
print(df_base.shape)
print(df_base.info())

(1011, 3)
(1011, 3)
(5022, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5022 entries, 0 to 1010
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rating  5022 non-null   float64
 1   title   5022 non-null   object 
 2   body    5022 non-null   object 
dtypes: float64(1), object(2)
memory usage: 156.9+ KB
None


In [ ]:
df_base.head()

,rating,title,body
0,5.0,Its good... tried and tested.,For its price its the best for iphone user.
1,5.0,Awesome Apple,"No comments, you just get it and Feel it☺️☺️"
2,5.0,So convenient for my Apple ecosystem at home,This was the one I was waiting for during the ...
3,5.0,Excellent...,Brilliant... thats what Apple is famous for......
4,5.0,Perfect Sound,Good battery life Small form factor One of the...


In [ ]:
## adding positive and negative tag corresponding to the data , 0:negative , 1 positive
def get_tag(x):
  tag=0
  if x>3:
    tag=1
  return tag
  
df_base['tag']=df_base['rating'].apply(get_tag)


In [ ]:
## Preprocessing Data
def text_clean(text_series):
    text_series = text_series.str.lower()
    clean_2 = text_series.str.replace(r"<.*>|[^a-zA-Z\s]","")
    clean_3 = clean_2.str.replace(r"\s+", " ")
    return clean_3
df_base["Text_Clean_title"] = text_clean(df_base["title"])
df_base["Text_Clean_body"] = text_clean(df_base["body"])

train_x, test_x, train_y, test_y = train_test_split(df_base["Text_Clean_title"], 
                                                    df_base["tag"], 
                                                    test_size = 0.3, 
                                                    random_state = 8)
df_base.drop(columns=['title','body'],inplace=True)


<ipython-input-146-008cf7e1e907>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_2 = text_series.str.replace(r"<.*>|[^a-zA-Z\s]","")
<ipython-input-146-008cf7e1e907>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_3 = clean_2.str.replace(r"\s+", " ")


In [ ]:
df_base.head()

,rating,tag,Text_Clean_title,Text_Clean_body
0,5.0,1,its good tried and tested,for its price its the best for iphone user
1,5.0,1,awesome apple,no comments you just get it and feel it
2,5.0,1,so convenient for my apple ecosystem at home,this was the one i was waiting for during the ...
3,5.0,1,excellent,brilliant thats what apple is famous for quality
4,5.0,1,perfect sound,good battery life small form factor one of the...


In [ ]:
## Converting text data into vector using bag of words 
Cnt_Vec = CountVectorizer(stop_words="english")
BOW_train = Cnt_Vec.fit_transform(train_x).toarray()
BOW_train_Df = pd.DataFrame(BOW_train, columns=Cnt_Vec.get_feature_names())
BOW_train_Df[:4]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absolutely,affordable,air,airpid,airpod,airpods,airpodsdelightful,airpords,airport,allowed,...,work,working,works,worst,worth,worthy,wow,wrost,year,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
BOW_test = Cnt_Vec.transform(test_x).toarray()
BOW_test_Df = pd.DataFrame(BOW_test, columns=Cnt_Vec.get_feature_names())
BOW_test_Df[:4]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absolutely,affordable,air,airpid,airpod,airpods,airpodsdelightful,airpords,airport,allowed,...,work,working,works,worst,worth,worthy,wow,wrost,year,zero
0,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
## Training
MNB_Model = MultinomialNB(alpha=1)
MNB_Model.fit(BOW_train, train_y)

MultinomialNB(alpha=1)

In [ ]:
## Predicting on test data
prediction = MNB_Model.predict(BOW_test)

In [ ]:
BOW_test.shape

(1507, 472)

In [ ]:
## confusion matrix for imbalanced dataset with 3 star reviews as well
predict_df=pd.DataFrame()
predict_df["final_category"]=prediction
predict_df["Original_Cateogry"]=test_y.to_list()
pd.crosstab(predict_df["final_category"], predict_df["Original_Cateogry"] )


Original_Cateogry,0,1
final_category,,
0,831,60
1,50,566


In [ ]:
## accuracy for imbalanced set of data
correct=0
for index,row in predict_df.iterrows():
  if row["final_category"]==row["Original_Cateogry"]:
     correct+=1

print("accuracy is",correct/predict_df.shape[0])

accuracy is 0.927007299270073


In [ ]:
## confusion matrix for balanced datashet
predict_df=pd.DataFrame()
predict_df["final_category"]=prediction
predict_df["Original_Cateogry"]=test_y.to_list()
pd.crosstab(predict_df["final_category"], predict_df["Original_Cateogry"] )

Original_Cateogry	0	1

final_category		
0	           280	     24

1          	6	       590

In [ ]:
## accuracy for balanced dataset with no 3 star review
correct=0
for index,row in predict_df.iterrows():
  if row["final_category"]==row["Original_Cateogry"]:
     correct+=1

print("accuracy is",correct/predict_df.shape[0])

accuracy is 0.9666666666666667